In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
attr = {      'StandardStats': 'stats_standard_11',
              'Goalkeeping': 'stats_keeper_adv_11',
              'Shooting': 'stats_shooting_11',
              'Passing': 'stats_passing_11',
              'Pass_types': 'stats_passing_types_11',
              'GCA': 'stats_gca_11',
              'Defense': 'stats_defense_11',
              'Possession': 'stats_possession_11',
              'Playing_time': 'stats_playing_time_11',
              'Misc': 'stats_misc_11'
             }

In [5]:
Leagues = { 'Serie A': 'https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats',
            'Premier League': 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats',
            'La Liga': 'https://fbref.com/en/comps/12/2022-2023/2022-2023-La-Liga-Stats',
            'Bundesliga': 'https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats',
            'Ligue 1': 'https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats'}

In [9]:
response = requests.get(Leagues['Serie A'])
response.status_code

200

In [11]:
Clubs = {}
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.select('#results2022-2023111_overall tbody')
    if table:
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        clubs = table_soup.find_all('tr')
        for row in clubs:
            Club = row.find('td', {'data-stat': 'team'}).text.strip()
            link = 'https://fbref.com/' + row.find('a')['href']
            Clubs[Club] = link
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)
print(Clubs)

{'Napoli': 'https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats', 'Lazio': 'https://fbref.com//en/squads/7213da33/2022-2023/Lazio-Stats', 'Inter': 'https://fbref.com//en/squads/d609edc0/2022-2023/Internazionale-Stats', 'Milan': 'https://fbref.com//en/squads/dc56fe14/2022-2023/Milan-Stats', 'Atalanta': 'https://fbref.com//en/squads/922493f3/2022-2023/Atalanta-Stats', 'Roma': 'https://fbref.com//en/squads/cf74a709/2022-2023/Roma-Stats', 'Juventus': 'https://fbref.com//en/squads/e0652b02/2022-2023/Juventus-Stats', 'Fiorentina': 'https://fbref.com//en/squads/421387cf/2022-2023/Fiorentina-Stats', 'Bologna': 'https://fbref.com//en/squads/1d8099f8/2022-2023/Bologna-Stats', 'Torino': 'https://fbref.com//en/squads/105360fe/2022-2023/Torino-Stats', 'Monza': 'https://fbref.com//en/squads/21680aa4/2022-2023/Monza-Stats', 'Udinese': 'https://fbref.com//en/squads/04eea015/2022-2023/Udinese-Stats', 'Sassuolo': 'https://fbref.com//en/squads/e2befd26/2022-2023/Sassuolo-Stats', 'Empoli': 'https

In [12]:
Clubs['Napoli']

'https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats'

In [40]:
def Scrap (table, club,keys):
    if table:
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        player_rows = table_soup.find_all('tr')
        player_data = []
        for row in player_rows:
            player_dict={}
            for name,key in keys.items():
                if key == 'club':
                    player_dict[name] = club
                elif key == 'league':
                    player_dict[name] = 'Serie A'
                elif key == 'player':
                    player_dict[name] = row.find('th', {'data-stat': key}).text.strip()
                elif key == 'nationality':
                    player_dict[name] = row.find('td', {'data-stat': key}).text.strip()[-3:]
                else:
                    player_dict[name] = row.find('td', {'data-stat': key}).text.strip()
            player_data.append(player_dict)
        df = pd.DataFrame(player_data)
    else:
        print("Table not found on the page.")
    return df

In [61]:
standard_stats = {'Name': 'player', 'Nationality' : 'nationality', 'Pos':'position','Age' : 'age','Club' : 'club',
                  'League' : 'league','MP' : 'games','Starts' : 'games_starts','Mins' : 'minutes','90s' : 'minutes_90s',
                  'Gls':'goals','Ast':'assists','G+A' : 'goals_assists','G-PK':'goals_pens','PKs' : 'pens_made',
                  'PKatt' : 'pens_att', 'Y':'cards_yellow','R':'cards_red','xG':'xg', 'npxG':'npxg','xAssistedG':'xg_assist',
                  'npxg_xAssistedG':'npxg_xg_assist', 'prgC':'progressive_carries','prgP':'progressive_passes',
                  'prgPR':'progressive_passes_received'}

advKeeper = {'Name': 'player', 'Nationality' : 'nationality', 'Pos':'position','Age' : 'age','Club' : 'club',
             'League' : 'league','90s' : 'minutes_90s','conceided':'gk_goals_against','PKc':'gk_pens_allowed',
             'FKc':'gk_free_kick_goals_against','CKc':'gk_corner_kick_goals_against','OwnGoal':'gk_own_goals_against',
             'PSxG':'gk_psxg','PSxG/SoT':'gk_psnpxg_per_shot_on_target_against','PSxG+/-':'gk_psxg_net','/90':'gk_psxg_net_per90',
             'PassC+40':'gk_passes_completed_launched','Passes att':'gk_passes_launched','Pass%':'gk_passes_pct_launched',
             'Passes':'gk_passes','Throws':'gk_passes_throws','Launch%':'gk_pct_passes_launched','AvgLen':'gk_passes_length_avg',
             'Goal kicks':'gk_goal_kicks','GKs Launched':'gk_pct_goal_kicks_launched','GKs AvgLen':'gk_goal_kick_length_avg',
             'Faced_crosses':'gk_crosses','Crosses_Stopped':'gk_crosses_stopped','Crosses_Stopped%':'gk_crosses_stopped_pct',
             'Action_Outside_Area':'gk_def_actions_outside_pen_area',
             'Action_Outside_Area/90':'gk_def_actions_outside_pen_area_per90','AvgDistOfAction':'gk_avg_distance_def_actions'}

shooting = {'Name': 'player', 'Nationality' : 'nationality', 'Pos':'position','Age' : 'age','Club' : 'club',
            'League' : 'league','90s' : 'minutes_90s','Goals':'goals','Shots':'shots','Shots_OT':'shots_on_target',
            'Shots_OT%':'shots_on_target_pct','Shots/90':'shots_per90','Shots_OT/90':'shots_on_target_per90',
            'Gs_per_shot':'goals_per_shot','G_per_ShotOT':'goals_per_shot_on_target', 'Avg_Shot_Dist':'average_shot_distance',
            'FK':'shots_free_kicks','PKs':'pens_made', 'PKatt':'pens_att','xG':'xg','npxG':'npxg','npxG_per_shot':'npxg_per_shot',
            'Goals-xG':'xg_net','npG-xG':'npxg_net'}

passing = {}
passing_type = {}
gca = {}
defense = {}
possession = {}
playing_time = {}
misc = {}


In [62]:
response = requests.get(Clubs['Napoli'])
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.select(f'table#stats_keeper_adv_11 tbody')

In [ ]:
response = requests.get(Clubs['Napoli'])
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.select(f'table#stats_standard_11 tbody')

In [ ]:
Standard_stats_df = Scrap(table,'Napoli',standard_stats)
clubs_data={}
Standard_stats_df

In [65]:
AdvKeeper_df = Scrap(table,'Napoli',advKeeper)
clubs_data={}
AdvKeeper_df

,Name,Nationality,Pos,Age,Club,League,90s,conceided,PKc,FKc,...,AvgLen,Goal kicks,GKs Launched,GKs AvgLen,Faced_crosses,Crosses_Stopped,Crosses_Stopped%,Action_Outside_Area,Action_Outside_Area/90,AvgDistOfAction
0,Alex Meret,ITA,GK,25,Napoli,Serie A,34.0,24,1,0,...,26.1,198,20.2,26.9,335,11,3.3,39,1.15,17.0
1,Pierluigi Gollini,ITA,GK,27,Napoli,Serie A,4.0,4,0,0,...,29.6,31,32.3,32.2,56,2,3.6,4,1.00,18.8


In [ ]:
for i,j in SerieA.items():
    response = requests.get(j)
    print(response.status_code)

In [ ]:
total_df = pd.concat(clubs_data.values(), ignore_index=True)

In [7]:
response = requests.get(SerieA['Napoli'])
print(response.status_code)

200


In [ ]:
StandardStats = {}
advKeeper = {}
shooting = {}
passing = {}
passing_type = {}
gca = {}
defense = {}
possession = {}
playing_time = {}
misc = {}

z = 0
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    for i,j in attr.items():
        table = soup.select(f'table#{j} tbody')
        print(z)
        z += 1
        Standard_st(table, 'Napoli')
